In [14]:
import sys
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from src.spectra import sensor_spectra
from src.bulk_parameters import Hs_band, compute_H0
from src.statistics import bin_stats



# 1. Scaling of Wave Setup

**Theory:** Use buckingham pi non-dimensional scaling to understand the $H_0^{1.75}L_0^{0.75}$ scaling. 

**Analytical Plan:**
- partition into discrete tidal bins (Low, mid, high)
- perform linear/nonlinear regression for each bin seperately
- try $\bar{\eta} = K * H_0^a*L_0^b*h_r^c$

**Deliverables:**
- Fig 1. Setup ($\bar{\eta}/H_0$) vs reef flat depth ($h_r$)
- Fig 2. test dependence on wave period
	- X: $H_0L_0$
	- Y: $\bar{\eta}/H_0$
- Fig 3. Setup vs Scaling Parameter
	- $\bar{\eta} = K * H_0^a*L_0^b*h_r^c$
	- or could be $\Pi = H_0^ah_r^c$   
    

In [21]:
binned = pd.read_parquet('../data_interim/binned2H0.parquet')
binned

,H0_mean,H0_std,H0_count,L0_mean,L0_std,L0_count,n_mean,n_std,n_count,H0L0_mean,...,steepness_count,setdown_mean,setdown_std,setdown_count,H0_sem,L0_sem,n_sem,H0L0_sem,steepness_sem,setdown_sem
0,0.576098,0.012128,8,119.378614,12.792140,8,-0.090554,0.011189,8,68.725396,...,8,-2.386840e-09,1.007913e-09,8,0.004288,4.522704,0.003956,2.467334,0.000223,3.563510e-10
1,0.658773,0.027603,77,135.119543,35.934925,77,-0.089376,0.013597,77,88.960679,...,77,-3.620465e-09,2.906515e-09,77,0.003146,4.095165,0.001550,2.720847,0.000134,3.312281e-10
2,0.751944,0.028076,117,121.206032,30.783891,117,-0.085279,0.017908,117,91.117089,...,117,-7.806208e-09,1.568104e-08,117,0.002596,2.845972,0.001656,2.149763,0.000120,1.449713e-09
3,0.848764,0.029844,102,104.655226,35.604861,102,-0.081251,0.018028,102,88.442928,...,102,-2.887473e-08,1.100392e-07,102,0.002955,3.525407,0.001785,2.881673,0.000264,1.089550e-08
4,0.952826,0.029349,173,90.151623,22.964306,173,-0.076940,0.019946,173,85.881294,...,173,-3.900670e-08,1.798346e-07,173,0.002231,1.745944,0.001516,1.685459,0.000199,1.367257e-08
5,1.055265,0.028332,225,86.174794,25.463671,225,-0.073213,0.022236,225,90.884935,...,225,-3.191663e-08,4.430222e-08,225,0.001889,1.697578,0.001482,1.788225,0.000232,2.953481e-09
6,1.150438,0.027783,288,88.608661,26.393417,288,-0.066402,0.024042,288,102.009721,...,288,-5.100543e-08,1.684694e-07,288,0.001637,1.555247,0.001417,1.814972,0.000228,9.927154e-09
7,1.249696,0.027884,255,94.323064,29.101052,255,-0.060472,0.032646,255,117.833796,...,255,-5.264471e-08,1.084458e-07,255,0.001746,1.822379,0.002044,2.279537,0.000248,6.791140e-09
8,1.351192,0.029285,251,89.791838,33.547060,251,-0.055082,0.032320,251,121.255913,...,251,-9.218028e-08,4.091683e-07,251,0.001848,2.117472,0.002040,2.847439,0.000292,2.582647e-08
9,1.449792,0.027489,279,91.099016,37.757313,279,-0.048991,0.037412,279,132.166005,...,279,-8.926420e-08,2.535240e-07,279,0.001646,2.260471,0.002240,3.305998,0.000308,1.517808e-08


In [ ]:
## Dataframe set up 

# load prior bulk_df
bulk_df = pd.read_parquet('../data_interim/bulk_df.parquet')

# Range of tide is from -0.47m to 0.51m
# defining "low tide" as <0 and high tide as >0
tide_df = bulk_df.copy()
tide_df['Tide_lo'] = tide_df['h_tide'] < 0
tide_df['Tide_hi'] = tide_df['h_tide'] > 0

,H0,n,h_tide,L0,H0/L0,H0L0,steepness,Tp,Hs_IG_Nearshore,setdown_x,setup,H6,setdown_y,setdown,Tide_lo,Tide_hi
time,,,,,,,,,,,,,,,,
2007-12-14 12:15:01.003921408,1.645071,-0.029936,0.173463,91.624145,0.017955,150.728218,0.017955,14.634210,0.196278,-8.612608e-08,-0.029936,1.786121,-8.612608e-08,-8.612608e-08,False,True
2007-12-14 12:30:01.007838208,1.626725,-0.078732,0.197391,82.591279,0.019696,134.353266,0.019696,13.846214,0.209341,-6.321125e-08,-0.078732,1.726074,-6.321125e-08,-6.321125e-08,False,True
2007-12-14 12:45:01.011755007,1.635449,-0.077717,0.220038,88.654160,0.018448,144.989335,0.018448,14.173290,0.210029,-6.381081e-08,-0.077717,1.752083,-6.381081e-08,-6.381081e-08,False,True
2007-12-14 13:00:01.015671808,1.512499,-0.017358,0.241270,76.812256,0.019691,116.178469,0.019691,13.333391,0.205247,-9.041383e-08,-0.017358,1.579092,-9.041383e-08,-9.041383e-08,False,True
2007-12-14 13:15:01.019588608,1.383515,-0.035062,0.260961,69.346901,0.019951,95.942475,0.019951,13.740518,0.168462,-8.536625e-08,-0.035062,1.463238,-8.536625e-08,-8.536625e-08,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-01-17 22:15:13.945028607,1.636291,0.074588,-0.382151,140.963989,0.011608,230.658171,0.011608,12.676112,0.256726,-7.538904e-08,0.074588,1.705907,-7.538904e-08,-7.538904e-08,True,False
2008-01-17 22:30:13.948945407,1.836326,0.065204,-0.398928,136.162355,0.013486,250.038532,0.013486,13.333391,0.227943,-7.189168e-08,0.065204,1.926385,-7.189168e-08,-7.189168e-08,True,False
2008-01-17 22:45:13.952862207,1.638650,0.045569,-0.414873,128.485516,0.012754,210.542782,0.012754,12.949697,0.248969,-7.355116e-08,0.045569,1.709467,-7.355116e-08,-7.355116e-08,True,False


In [22]:
tide_df.keys()

Index(['H0', 'n', 'h_tide', 'L0', 'H0/L0', 'H0L0', 'steepness', 'Tp',
       'Hs_IG_Nearshore', 'setdown_x', 'setup', 'H6', 'setdown_y', 'setdown',
       'Tide_lo', 'Tide_hi'],
      dtype='object')